In [1]:
import pandas as pd
import mysql.connector as sql
import pymongo as py

In [2]:
client=py.MongoClient('mongodb://localhost:27017')
db=client['AirBNB']

### 1.Room DataFrame

In [3]:
data_List1=[]

for i in db.Data.find( {}, {'_id':1,'name':1,'property_type':1,'room_type':1,'bed_type':1,
                        'minimum_nights':1,'maximum_nights':1,'cancellation_policy':1,'accommodates':1,
                        'bedrooms':1,'beds':1,'number_of_reviews':1,'bathrooms':1,'price':1,
                        'cleaning_fee':1,'extra_people':1,'guests_included':1,
                        'review_scores.review_scores_rating':1} ):
    data_List1.append(i)

df_ROOMS=pd.DataFrame(data_List1)
# print(df_ROOMS.head())

In [4]:
print("Total No. oF Null Values in the DataFrame")
print(df_ROOMS.isnull().sum())

Total No. oF Null Values in the DataFrame
_id                       0
name                      0
property_type             0
room_type                 0
bed_type                  0
minimum_nights            0
maximum_nights            0
cancellation_policy       0
accommodates              0
bedrooms                  5
beds                     13
number_of_reviews         0
bathrooms                10
price                     0
cleaning_fee           1531
extra_people              0
guests_included           0
review_scores             0
dtype: int64


In [5]:
df_ROOMS.fillna({'bedrooms':0},inplace=True)
df_ROOMS.fillna({'beds':0},inplace=True)
df_ROOMS.fillna({'bathrooms':0},inplace=True)
df_ROOMS.fillna({'cleaning_fee':0},inplace=True)
df_ROOMS['review_scores'] = df_ROOMS['review_scores'].apply(lambda x: x.get('review_scores_rating',0))
print(df_ROOMS.isnull().sum())

_id                    0
name                   0
property_type          0
room_type              0
bed_type               0
minimum_nights         0
maximum_nights         0
cancellation_policy    0
accommodates           0
bedrooms               0
beds                   0
number_of_reviews      0
bathrooms              0
price                  0
cleaning_fee           0
extra_people           0
guests_included        0
review_scores          0
dtype: int64


In [6]:
for Col in df_ROOMS.columns:
    print("Column Name:",Col)
    print(df_ROOMS[Col].unique())
    print("************************")

Column Name: _id
['1003530' '10133554' '10059872' ... '9887054' '9907907' '9936016']
************************
Column Name: name
['New York City - Upper West Side Apt'
 'Double and triple rooms Blue mosque'
 'Soho Cozy, Spacious and Convenient' ...
 'Quarry Bay fully furnished studio C' 'Large double room in Bondi'
 'Alugo Apartamento na Tijuca']
************************
Column Name: property_type
['Apartment' 'Bed and breakfast' 'Guesthouse' 'Hostel'
 'Serviced apartment' 'Loft' 'House' 'Condominium' 'Treehouse'
 'Guest suite' 'Bungalow' 'Townhouse' 'Villa' 'Cabin' 'Other' 'Farm stay'
 'Chalet' 'Boutique hotel' 'Cottage' 'Boat' 'Earth house' 'Aparthotel'
 'Resort' 'Tiny house' 'Nature lodge' 'Hotel' 'Camper/RV'
 'Casa particular (Cuba)' 'Barn' 'Hut' 'Heritage hotel (India)'
 'Pension (South Korea)' 'Campsite' 'Castle' 'Houseboat' 'Train']
************************
Column Name: room_type
['Private room' 'Entire home/apt' 'Shared room']
************************
Column Name: bed_type
['Rea

In [7]:
df_ROOMS['minimum_nights']=df_ROOMS['minimum_nights'].astype(int)
df_ROOMS['maximum_nights']=df_ROOMS['maximum_nights'].astype(int)
df_ROOMS['bedrooms']=df_ROOMS['bedrooms'].astype(int)
df_ROOMS['beds']=df_ROOMS['beds'].astype(int)
df_ROOMS['bathrooms']=df_ROOMS['bathrooms'].astype(int)
df_ROOMS['price']=df_ROOMS['price'].astype(int)
df_ROOMS['cleaning_fee']=df_ROOMS['cleaning_fee'].apply(lambda x: int(x) if x!=0 else 'Not Specified')

In [ ]:
print(df_ROOMS.head())

        _id                                              name  \
0   1003530               New York City - Upper West Side Apt   
1  10133554               Double and triple rooms Blue mosque   
2  10059872                Soho Cozy, Spacious and Convenient   
3  10084023                 City center private room with bed   
4  10091713  Surry Hills Studio - Your Perfect Base in Sydney   

       property_type        room_type  bed_type  minimum_nights  \
0          Apartment     Private room  Real Bed              12   
1  Bed and breakfast     Private room  Real Bed               1   
2          Apartment  Entire home/apt  Real Bed               4   
3         Guesthouse     Private room     Futon               1   
4          Apartment  Entire home/apt  Real Bed              10   

   maximum_nights          cancellation_policy  accommodates  bedrooms  beds  \
0             360  strict_14_with_grace_period             2         1     1   
1            1125                     moderate

### 2.Host DataFrame

In [9]:
data_LIST2=[]
for i in db.Data.find({},{'id':1,'host':1}):
        data_LIST2.append(i)

#Removing Unneccessary Data
df_HOST= pd.DataFrame(data_LIST2)
host_keys = list(df_HOST.iloc[0,1].keys())
host_keys.remove('host_about')

for i in host_keys:
    if i == 'host_response_time':
        df_HOST['host_response_time'] = df_HOST['host'].apply(lambda x: x['host_response_time'] if 'host_response_time' in x else 'Not Specified')
    else:
        df_HOST[i] = df_HOST['host'].apply(lambda x: x[i] if i in x and x[i]!='' else 'Not Specified')

df_HOST.drop(columns=['host'], inplace=True)

df_HOST[df_HOST['_id']=='10038496']
df_HOST.drop(columns=['host_url','host_thumbnail_url','host_picture_url','host_is_superhost','host_has_profile_pic','host_listings_count','host_verifications'],axis=1,inplace=True)
# df_HOST

In [10]:
print(df_HOST.head())

        _id   host_id    host_name                      host_location  \
0   1003530    454250        Greta  New York, New York, United States   
1  10133554  52004703  Mehmet Emin         Istanbul, İstanbul, Turkey   
2  10059872  51624384     Giovanni               Hong Kong, Hong Kong   
3  10084023  51744313           Yi                      United States   
4  10091713  13764143          Ben         New South Wales, Australia   

   host_response_time host_neighbourhood host_response_rate  \
0      within an hour      Not Specified                100   
1  within a few hours      Not Specified                100   
2       Not Specified               Soho      Not Specified   
3      within an hour       Shek Kip Mei                100   
4       Not Specified        Surry Hills      Not Specified   

   host_identity_verified  host_total_listings_count  
0                    True                          3  
1                    True                          2  
2                

In [11]:
print(df_HOST.isnull().sum())

_id                          0
host_id                      0
host_name                    0
host_location                0
host_response_time           0
host_neighbourhood           0
host_response_rate           0
host_identity_verified       0
host_total_listings_count    0
dtype: int64


### 3.Address DataFrame

In [12]:
data_LIST3=[]
for i in db.Data.find({},{'id':1,'address':1}):
    data_LIST3.append(i)

df_ADDRESS=pd.DataFrame(data_LIST3)
address_KEYS=list(df_ADDRESS.iloc[0,1].keys())


# address_KEYS

#Now we are going to split the address row into different row

for i in address_KEYS:
    if i == 'location':
        df_ADDRESS['location_type'] = df_ADDRESS['address'].apply(lambda x: x['location']['type'])
        df_ADDRESS['longitude'] = df_ADDRESS['address'].apply(lambda x: x['location']['coordinates'][0])
        df_ADDRESS['latitude'] = df_ADDRESS['address'].apply(lambda x: x['location']['coordinates'][1])
        #df_ADDRESS['is_location_exact'] = df_ADDRESS['address'].apply(lambda x: x['location']['is_location_exact'])
    else:
        df_ADDRESS[i] = df_ADDRESS['address'].apply(lambda x: x[i] if x[i]!='' else 'Not Specified')

df_ADDRESS.drop(columns=['address','street'], inplace=True)


In [13]:
print(df_ADDRESS.isnull().sum())

_id                0
suburb             0
government_area    0
market             0
country            0
country_code       0
location_type      0
longitude          0
latitude           0
dtype: int64


In [14]:
print(df_ADDRESS.head())

        _id                      suburb    government_area     market  \
0   1003530                   Manhattan    Upper West Side   New York   
1  10133554                       Fatih              Fatih   Istanbul   
2  10059872  Central & Western District  Central & Western  Hong Kong   
3  10084023       Sham Shui Po District       Sham Shui Po  Hong Kong   
4  10091713                Darlinghurst             Sydney     Sydney   

         country country_code location_type  longitude  latitude  
0  United States           US         Point  -73.96523  40.79962  
1         Turkey           TR         Point   28.98009  41.00620  
2      Hong Kong           HK         Point  114.15027  22.28158  
3      Hong Kong           HK         Point  114.16690  22.33140  
4      Australia           AU         Point  151.21554 -33.88029  


### 4.Availability DataFrame

In [15]:
availability = []
for i in db.Data.find( {}, {'_id':1, 'availability':1}):
    availability.append(i)

df_AVAILABILITY = pd.DataFrame(availability)
availability_keys = list(df_AVAILABILITY.iloc[0,1].keys())


for i in availability_keys:
    df_AVAILABILITY['availability_30'] = df_AVAILABILITY['availability'].apply(lambda x: x['availability_30'])
    df_AVAILABILITY['availability_60'] = df_AVAILABILITY['availability'].apply(lambda x: x['availability_60'])
    df_AVAILABILITY['availability_90'] = df_AVAILABILITY['availability'].apply(lambda x: x['availability_90'])
    df_AVAILABILITY['availability_365'] = df_AVAILABILITY['availability'].apply(lambda x: x['availability_365'])

df_AVAILABILITY.drop(columns=['availability'], inplace=True)
print(df_AVAILABILITY.head())

        _id  availability_30  availability_60  availability_90  \
0   1003530                0                0                0   
1  10133554               30               60               90   
2  10059872                0                0                0   
3  10084023               14               24               40   
4  10091713                0                0                0   

   availability_365  
0                93  
1               365  
2                 0  
3               220  
4                 0  


In [16]:
print(df_AVAILABILITY.isnull().sum())

_id                 0
availability_30     0
availability_60     0
availability_90     0
availability_365    0
dtype: int64


### 5. Amenties DataFrame

In [17]:
def amenities_SORT(x):
    a = x
    a.sort(reverse=False)
    return a

data_LIST5 = []
for i in db.Data.find( {}, {'_id':1, 'amenities':1}):
    data_LIST5.append(i)

df_AMENITIES = pd.DataFrame(data_LIST5)
df_AMENITIES['amenities'] = df_AMENITIES['amenities'].apply(lambda x: amenities_SORT(x))


In [18]:
print(df_AMENITIES.head())

        _id                                          amenities
0   1003530  [Air conditioning, Buzzer/wireless intercom, D...
1  10133554  [Air conditioning, Building staff, Dryer, Esse...
2  10059872  [24-hour check-in, Air conditioning, Doorman, ...
3  10084023  [Air conditioning, Dishes and silverware, Elev...
4  10091713  [Bed linens, Dryer, Elevator, Essentials, Ethe...


In [19]:
print(df_AMENITIES.isnull().sum())

_id          0
amenities    0
dtype: int64


In [20]:
df = pd.merge(df_ROOMS, df_HOST, on='_id')
df = pd.merge(df, df_ADDRESS, on='_id')
df = pd.merge(df, df_AVAILABILITY, on='_id')
df=pd.merge(df, df_AMENITIES, on='_id')
#df.to_csv('Cleaned_Dataset.csv', index=False)